In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def make_env():
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium.

    Questa funzione istanzia l'ambiente "Ant-v5", uno degli ambienti recenti e ben supportati
    in Gymnasium. I parametri usati sono:
    - reset_noise_scale (0.1): determina la scala del rumore quando l'ambiente viene resettato.
    - render_mode ('None'): indica che non verrà effettuato il rendering durante l'esecuzione.

    Ritorna:
        gym.Env: l'ambiente Ant-v5 inizializzato.
    """
    
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make(
        "Ant-v5",
        reset_noise_scale=0.07467935946049155,
        forward_reward_weight=0.07619282247649807,
        ctrl_cost_weight=0.9714884866781529,
        healthy_reward=0.7882423009290584,
        render_mode='None'
)

In [3]:
# 1. Creiamo un ambiente vettorializzato (Vectorized Environment)
# Utilizziamo DummyVecEnv per gestire più istanze dell'ambiente come se fossero una singola entità.
# Qui passiamo la funzione make_env (definita in un'altra cella) che crea l'ambiente "Ant-v5".
env = DummyVecEnv([make_env])  

# 2. Normalizziamo osservazioni (obs) e ricompense (reward)
# VecNormalize scala le osservazioni e le ricompense per stabilizzare l'allenamento.
# Parametri:
#   norm_obs=True   -> Abilita la normalizzazione delle osservazioni.
#   norm_reward=True -> Abilita la normalizzazione delle ricompense.
#   clip_obs=10.     -> Limita i valori normalizzati dell'osservazione a un range [-10, 10] per evitare estremi.
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='None' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(


In [4]:
# 3. Definiamo il modello RL (PPO) con spiegazioni dettagliate per ciascun parametro

model = PPO(
    policy="MlpPolicy",           # Tipo di policy: una rete neurale MLP (Multilayer Perceptron) che mappa osservazioni ad azioni
    env=env,                      # Ambiente di addestramento: usa l'ambiente vettorializzato e normalizzato creato in precedenza
    learning_rate=0.0005596010992131322,           # Tasso di apprendimento: controlla la velocità con cui il modello apprende aggiornando i pesi
    n_steps=2048,                 # Numero di passi da eseguire nell'ambiente per ogni ciclo di aggiornamento della policy
    batch_size=256,                # Dimensione del batch per gli aggiornamenti stocastici: suddivide i dati raccolti nei mini-batch
    n_epochs=10,                  # Numero di volte (epoch) che il dataset raccolto viene utilizzato per aggiornare la policy
    gamma=0.91848669540811,      # Fattore di sconto: determina l'importanza delle ricompense future rispetto a quelle immediate
    gae_lambda=0.95,              # Parametro per il Generalized Advantage Estimation (GAE): bilancia bias e varianza nella stima dell'advantage
    clip_range=0.17,               # Intervallo di clipping: limita le variazioni della policy per mantenere aggiornamenti stabili
    ent_coef=0.06,                 # Coefficiente di entropia: controlla l'incentivo all'esplorazione; 0 significa nessun bonus per l'entropia
    verbose=1,                    # Livello di verbosità: 1 per stampare informazioni di log utili durante l'addestramento
    tensorboard_log="./ppo_Ant_tensorboard/",  # Cartella per salvare i log di TensorBoard
    device='mps'                    # Specifica l'uso della GPU su Apple Silicon
)
#PPO_3 lr=0.0005596010992131322 gamma=0.91848669540811 ent_coef=0.09 n_steps=2048 batch_size=256 n_epochs=10 clip_range=0.15 gae_lambda=0.95

Using mps device


/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [5]:
# 4. Alleniamo il modello
# Il parametro total_timesteps indica il numero totale di iterazioni (o passi)
# che il modello eseguirà durante l'allenamento. Ogni timestep rappresenta un'interazione
# con l'ambiente in cui il modello esegue un'azione e riceve un feedback, che viene poi
# usato per aggiornare la sua politica interna.
total_timesteps = 500000  # Puoi aumentare questo valore per permettere al modello di acquisire più esperienza.
model.learn(total_timesteps=total_timesteps)

Logging to ./ppo_Ant_tensorboard/PPO_4
-----------------------------
| time/              |      |
|    fps             | 384  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 357         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017471708 |
|    clip_fraction        | 0.187       |
|    clip_range           | 0.17        |
|    entropy_loss         | -11.4       |
|    explained_variance   | -1.65       |
|    learning_rate        | 0.00056     |
|    loss                 | -0.72       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0312     |
|    std                  | 1.02        |
|    value_loss           | 0.281    

KeyboardInterrupt: 

In [ ]:
#5. Salviamo il modello
model.save("ppo_Ant_model")
env.save("vecnormalize_Ant.pkl")  # salviamo anche i parametri di normalizzazione

In [ ]:
def evaluate_policy(env, policy, episodes=50):
    """
    Valuta una policy addestrata su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - policy: La policy addestrata da valutare.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
        done = False
        total_reward = 0
        while not done:
            action, _ = policy.predict(obs)  # Predice l'azione da eseguire
            obs, reward, done, _ = env.step(action)  # Esegue l'azione e ottiene il feedback dall'ambiente
            total_reward += reward  # Accumula la ricompensa ottenuta
        total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
    return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense


def evaluate_random_policy(env, episodes=50):
    """
    Valuta una policy casuale su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
        done = False
        total_reward = 0
        while not done:
            action = env.action_space.sample()  # Genera un'azione casuale
            if isinstance(action, np.ndarray):
                action = action.flatten()
            obs, reward, done, _ = env.step(action)  # Esegue l'azione e ottiene il feedback dall'ambiente
            total_reward += reward  # Accumula la ricompensa ottenuta
        total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
    return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense

In [ ]:
# Valutazione dopo l'addestramento
mean_reward_trained, std_reward_trained = evaluate_policy(env, model)  # Valuta la policy addestrata
mean_reward_random, std_reward_random = evaluate_random_policy(env)  # Valuta la policy casuale

# Stampa dei risultati
print(f"Trained Policy: Mean Reward: {mean_reward_trained}, Std: {std_reward_trained}")
print(f"Random Policy: Mean Reward: {mean_reward_random}, Std: {std_reward_random}")

# Creazione del grafico di confronto
labels = ['Random Policy', 'Trained Policy']
means = [mean_reward_random, mean_reward_trained]
stds = [std_reward_random, std_reward_trained]

plt.figure(figsize=(8, 5))
plt.bar(labels, means, yerr=stds, capsize=10, color=['skyblue', 'lightgreen'])
plt.ylabel('Mean Episodic Reward')
plt.title('Policy Comparison')
plt.show()

NameError: name 'env' is not defined